## Principales Runnables en LCEL

Entro los principales Runnables que nos otorga LangChain, tenemos:
1. RunnablePassthrough: No altera el dato de entrada, útil para enviar datos sin procesamiento.
2. RunnableLambda: Ejecuta una función personalizada, ideal para lógica específica.
3. RunnableParallel: Ejecuta múltiples Runnables en paralelo, aumentando eficiencia.
4. RunnableBranch: Selecciona Runnables según condiciones, útil para bifurcaciones en el flujo de trabajo.

In [10]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']
model_llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

## RunnablePassthrough
* No hace nada con los datos de entrada.
* Veámoslo con un ejemplo muy simple: una cadena con solo 'RunnablePassthrough()' generará la entrada original sin ninguna modificación.

In [2]:
chain = RunnablePassthrough()
chain.invoke("Abram")

'Abram'

## RunnableLambda
* Para utilizar una función personalizada dentro de una cadena LCEL, debemos encapsularla con RunnableLambda.
* Definamos una función muy simple para crear apellidos rusos:

In [4]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("Abram")

'Abramovich'

## RunnableParallel
* Usaremos RunnableParallel() para ejecutar tareas en paralelo.
* Este es probablemente el Runnable más importante y útil de LangChain.
* En la siguiente cadena, RunnableParallel ejecutará estas dos tareas en paralelo:
* operación_a usará RunnablePassthrough.
* operación_b usará RunnableLambda con la función russian_lastname.

In [7]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

chain.invoke("Abram")

{'operation_a': 'Abram', 'operation_b': 'Abramovich'}

* En lugar de utilizar RunnableLambda, ahora vamos a utilizar una función lambda e invocaremos la cadena con dos entradas:

In [9]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": lambda x: x["name"]+"ovich"
    }
)

chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

{'operation_a': {'name1': 'Jordam', 'name': 'Abram'},
 'soccer_player': 'Abramovich'}

#### Podemos agregar más Runnables a la cadena
* En el siguiente ejemplo, el indicador Runnable tomará la salida de RunnableParallel:

In [10]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [11]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model_llm | output_parser


chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'Roman Abramovich, the Russian billionaire and former owner of Chelsea Football Club, is known not just for his wealth and business ventures but also for his unique collection of art. One particularly curious fact about him is that he owns one of the largest private collections of contemporary art in the world, including works by renowned artists such as Francis Bacon, Lucian Freud, and Damien Hirst. His passion for art is reflected in his investments and his involvement in various cultural initiatives, showcasing a side of him that goes beyond his business acumen and sports interests.'

* Como viste, el indicador Runnable tomó "Abramovich", la salida de RunnableParallel, como valor para la variable "soccer_player".

## Veamos un uso más avanzado de RunnableParallel

Es necesario que instalemos la siguiente librería:
```
poetry add faiss-cpu
```
¿Qué hace esta librería?


In [1]:
from langchain_community.vectorstores import FAISS


In [4]:
# Creamos una base de datos vectorial usando el motor de búsqueda FAISS
vectorstore = FAISS.from_texts(
    ["AI Accelera ha formado a más de 10.000 antiguos alumnos de todos los continentes y de las mejores empresas"], embedding=OpenAIEmbeddings()
)

In [7]:
retriever = vectorstore.as_retriever()

template = """Responde a la pregunta basada en el siguiente contexto: {context}
Pregunta: {question}
"""

# Armamos el prompt
prompt = ChatPromptTemplate.from_template(template)


retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model_llm
    | output_parser
)

retrieval_chain.invoke("¿Quienes son los alumnos de AI Accelera?")


'Los alumnos de AI Accelera son más de 10.000 antiguos alumnos que provienen de todos los continentes y de las mejores empresas.'

### Importante: la sintaxis de RunnableParallel puede tener varias variaciones.
Al componer un RunnableParallel con otro Runnable, no es necesario encapsularlo en la clase RunnableParallel. Dentro de una cadena, las siguientes tres sintaxis son equivalentes:
* `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
* `RunnableParallel(context=retriever, question=RunnablePassthrough())`
* `{"context": retriever, "question": RunnablePassthrough()}`

## Uso de itemgetter con RunnableParallel
* Cuando se llama al LLM con varias variables de entrada diferentes.

In [8]:
from operator import itemgetter

In [13]:
vectorstore = FAISS.from_texts(
    ["AI Accelera ha capacitado a más de 5.000 antiguos alumnos de Enterprise."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Responde la pregunta basada solo en el siguiente contexto: {context}

Pregunta: {question}

Responde en el siguiente lenguaje: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model_llm
    | output_parser
)

chain.invoke({"question": "Cuantas personlas han sido entrenadas por AI Accelera?", "language": "Español Pirata"})

'¡Argh! AI Accelera ha entrenado a más de 5.000 viejos marineros, ¡eso es un montón de almas!'

## RunnableBranch: Router Chain
* Una RunnableBranch es un tipo especial de ejecutable que le permite definir un conjunto de condiciones y ejecutables para ejecutar en función de la entrada.
* **Una RunnableBranch se inicializa con una lista de pares (condición, ejecutable) y un ejecutable predeterminado**. Selecciona qué rama pasa a cada condición la entrada con la que se invoca. Selecciona la primera condición que se evalúa como Verdadera y ejecuta el ejecutable correspondiente a esa condición con la entrada.
* Para usos avanzados, una [función personalizada](https://python.langchain.com/v0.1/docs/expression_language/how_to/routing/) puede ser una mejor alternativa que RunnableBranch.

El siguiente ejemplo avanzado puede clasificar y responder a las preguntas de los usuarios en función de temas específicos, como rock, política, historia, deportes o consultas generales. **Utiliza algunos temas nuevos que explicaremos en la siguiente lección**. A continuación, se incluye una explicación simplificada de cada parte:

1. **Plantillas de indicaciones**: cada plantilla está diseñada para un tema específico:
- **rock_template**: configurada para preguntas relacionadas con el rock and roll.
- **politics_template**: diseñada para responder preguntas sobre política.
- **history_template**: diseñada para consultas relacionadas con la historia.
- **sports_template**: configurada para responder preguntas relacionadas con los deportes.
- **general_prompt**: una plantilla general para consultas que no se ajustan a las categorías específicas.

Cada plantilla incluye un marcador de posición `{input}` donde se insertará la pregunta real del usuario.

2. **RunnableBranch**: este es un mecanismo de ramificación que selecciona qué plantilla usar en función del tema de la pregunta. Evalúa condiciones (como `x["topic"] == "rock"`) para determinar el tema y utiliza la plantilla de solicitud adecuada.

3. **Clasificador de temas**: una clase de Pydantic que clasifica el tema de la pregunta de un usuario en una de las categorías predefinidas (rock, política, historia, deportes o general).

4. **Cadena de clasificadores**:
- **Cadena**: procesa la entrada del usuario para predecir el tema.
- **Analizador**: extrae el tema predicho de la salida del clasificador.

5. **RunnablePassthrough**: este componente introduce la entrada del usuario y el tema clasificado en RunnableBranch.

6. **Cadena final**:
- La entrada del usuario se procesa primero para clasificar su tema.
- Luego, se selecciona la solicitud adecuada en función del tema clasificado.
- La solicitud seleccionada se utiliza para formular una pregunta que luego se envía a un modelo (como ChatOpenAI).
- La respuesta del modelo se analiza como una cadena y se devuelve.

7. **Ejecución**:
- Se invoca la cadena con una pregunta de muestra, "¿Quién fue Napoleón Bonaparte?".
- En función de la clasificación, selecciona la plantilla adecuada, genera una consulta al modelo de chat y procesa la respuesta.

El sistema crea efectivamente un generador de respuestas dinámicas que ajusta la forma en que responde en función del tema de la consulta, haciendo uso de conocimientos especializados para diferentes temas.

In [14]:
from langchain.prompts import PromptTemplate

In [16]:
rock_template = """Eres un profesor de rock and roll muy inteligente. \
Eres muy bueno respondiendo preguntas sobre rock and roll de una manera concisa\
y fácil de entender.

Aquí tienes una pregunta:
{input}"""

rock_prompt = PromptTemplate.from_template(rock_template)

politics_template = """Eres un profesor de política muy bueno. \
Eres muy bueno respondiendo preguntas sobre política..

Aquí tienes una pregunta:
{input}"""

politics_prompt = PromptTemplate.from_template(politics_template)

history_template = """Eres un profesor de historia muy bueno. \
Tienes un excelente conocimiento y comprensión de personas,\
eventos y contextos de una variedad de períodos históricos.

Aquí tienes una pregunta:
{input}"""

history_prompt = PromptTemplate.from_template(history_template)

sports_template = """ Eres profesor de deportes.\
Eres muy bueno respondiendo preguntas sobre deportes.

Aquí tienes una pregunta:
{input}"""

sports_prompt = PromptTemplate.from_template(sports_template)

In [17]:
from langchain.schema.runnable import RunnableBranch

In [18]:
general_prompt = PromptTemplate.from_template(
"Eres un asistente útil. Responde la pregunta con la mayor precisión posible.\n\n{input}"
)

prompt_branch = RunnableBranch(
  (lambda x: x["topic"] == "rock", rock_prompt),
  (lambda x: x["topic"] == "politics", politics_prompt),
  (lambda x: x["topic"] == "history", history_prompt),
  (lambda x: x["topic"] == "sports", sports_prompt),
  general_prompt
)

In [20]:
from typing import Literal
from pydantic import BaseModel

from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain_core.utils.function_calling import convert_to_openai_function


class TopicClassifier(BaseModel):
    "Clasificar el tema de la pregunta del usuario"

    topic: Literal["rock", "politics", "history", "sports"]
    "El tema de la pregunta del usuario. Uno de 'rock', 'politics', 'history', 'sports' o 'general'."

classifier_function = convert_to_openai_function(TopicClassifier)

llm = ChatOpenAI().bind(functions=[classifier_function], function_call={"name": "TopicClassifier"})

parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = llm | parser

La función `classifier_function` clasifica o categoriza el tema de la pregunta de un usuario en categorías específicas como "rock", "política", "historia" o "deportes". Así es como funciona en términos simples:

1. **Conversión a función**: Convierte la clase Pydantic `TopicClassifier`, que es un sistema de clasificación predefinido, en una función que se puede usar fácilmente con LangChain. Este proceso de conversión implica encapsular la clase para que pueda integrarse y ejecutarse dentro de un modelo OpenAI.

2. **Detección de tema**: Cuando ingresa una pregunta, esta función analiza el contenido de la pregunta para determinar a qué categoría o tema pertenece. Busca palabras clave o patrones que coincidan con temas específicos. Por ejemplo, si la pregunta es sobre una banda de rock, el clasificador identificaría el tema como "rock".

3. **Salida**: La función genera el tema identificado como una etiqueta simple, como "rock" o "historia". Esta etiqueta es utilizada por otras partes de LangChain para decidir cómo manejar la pregunta, como elegir la plantilla correcta para formular una respuesta.

En esencia, la `classifier_function` actúa como un filtro inteligente que ayuda al sistema a entender qué tipo de pregunta se está haciendo para que pueda responder de manera más precisa y relevante.

In [ ]:
final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | ChatOpenAI()
    | StrOutputParser()
)

In [22]:
final_chain.invoke(
    {"input": "¿Quién es Napoleón Bonaparte?"}
)

'Napoleón Bonaparte fue un líder militar y político francés que se convirtió en emperador de Francia en el siglo XIX. Nacido en Córcega en 1769, Napoleón ascendió rápidamente en las filas del ejército durante la Revolución Francesa y se convirtió en líder militar durante las Guerras Napoleónicas. Conocido por su genio militar y su ambición política, Napoleón conquistó gran parte de Europa occidental y central, estableciendo un vasto imperio francés. Sin embargo, su imperio eventualmente colapsó y fue derrotado en la Batalla de Waterloo en 1815. A pesar de su caída, Napoleón sigue siendo una figura influyente en la historia europea y mundial.'